In [ ]:
from __future__ import print_function, division
import scipy

from keras.datasets import mnist
from keras_contrib.layers.normalization import InstanceNormalization
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
import datetime
import matplotlib.pyplot as plt
from data_loader import DataLoader
import numpy as np
import os



## sigin
import signal
import sys

In [ ]:
## source from keras-gan

class colorizeSketch():
    def __init__(self):
        # Input shape
        self.img_rows = 128
        self.img_cols = 128
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)

        # Configure data loader
        self.dataset_name = 'anime'
        self.data_loader = DataLoader(dataset_name=self.dataset_name,
                                      img_res=(self.img_rows, self.img_cols))


        # Calculate output shape of D (PatchGAN)
        patch = int(self.img_rows / 2**4)
        self.disc_patch = (patch, patch, 1)

        # Number of filters in the first layer of G and D
        self.gf = 64
        self.df = 64

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminators
        self.d_A = self.build_discriminator()
        self.d_B = self.build_discriminator()
        self.d_A.compile(loss='mse',
            optimizer=optimizer,
            metrics=['accuracy'])
        self.d_B.compile(loss='mse',
            optimizer=optimizer,
            metrics=['accuracy'])

        #-------------------------
        # Construct Computational
        #   Graph of Generators
        #-------------------------

        # Build the generators
        self.g_AB = self.build_generator()
        self.g_BA = self.build_generator()

        # Input images from both domains
        img_A = Input(shape=self.img_shape)
        img_B = Input(shape=self.img_shape)

        # Translate images to the other domain
        fake_B = self.g_AB(img_A)
        fake_A = self.g_BA(img_B)
        # Translate images back to original domain
        reconstr_A = self.g_BA(fake_B)
        reconstr_B = self.g_AB(fake_A)

        # For the combined model we will only train the generators
        self.d_A.trainable = True
        self.d_B.trainable = True

        # Discriminators determines validity of translated images
        valid_A = self.d_A(fake_A)
        valid_B = self.d_B(fake_B)

        # Objectives
        # + Adversarial: Fool domain discriminators
        # + Translation: Minimize MAE between e.g. fake B and true B
        # + Cycle-consistency: Minimize MAE between reconstructed images and original
        self.combined = Model(inputs=[img_A, img_B],
                              outputs=[ valid_A, valid_B,
                                        fake_B, fake_A,
                                        reconstr_A, reconstr_B ])
        self.combined.compile(loss=['mse', 'mse',
                                    'mae', 'mae',
                                    'mae', 'mae'],
                              optimizer=optimizer)
        

    def build_generator(self):
        """U-Net Generator"""

        def conv2d(layer_input, filters, f_size=4, normalize=True):
            """Layers used during downsampling"""
            d = Conv2D(filters, kernel_size=f_size, strides=2, padding='same')(layer_input)
            d = LeakyReLU(alpha=0.2)(d)
            if normalize:
                d = InstanceNormalization()(d)
            return d

        def deconv2d(layer_input, skip_input, filters, f_size=4, dropout_rate=0):
            """Layers used during upsampling"""
            u = UpSampling2D(size=2)(layer_input)
            u = Conv2D(filters, kernel_size=f_size, strides=1, padding='same', activation='relu')(u)
            if dropout_rate:
                u = Dropout(dropout_rate)(u)
            u = InstanceNormalization()(u)
            u = Concatenate()([u, skip_input])
            return u

        # Image input
        d0 = Input(shape=self.img_shape)

        # Downsampling
        d1 = conv2d(d0, self.gf, normalize=False)
        d2 = conv2d(d1, self.gf*2)
        d3 = conv2d(d2, self.gf*4)
        d4 = conv2d(d3, self.gf*8)
        d5 = conv2d(d4, self.gf*8)
        d6 = conv2d(d5, self.gf*8)
        d7 = conv2d(d6, self.gf*8)

        # Upsampling
        u1 = deconv2d(d7, d6, self.gf*8)
        u2 = deconv2d(u1, d5, self.gf*8)
        u3 = deconv2d(u2, d4, self.gf*8)
        u4 = deconv2d(u3, d3, self.gf*4)
        u5 = deconv2d(u4, d2, self.gf*2)
        u6 = deconv2d(u5, d1, self.gf)

        u7 = UpSampling2D(size=2)(u6)
        output_img = Conv2D(self.channels, kernel_size=4, strides=1,
                            padding='same', activation='tanh')(u7)

        return Model(d0, output_img)

    def build_discriminator(self):

        def d_layer(layer_input, filters, f_size=4, normalization=True):
            """Discriminator layer"""
            d = Conv2D(filters, kernel_size=f_size, strides=2, padding='same')(layer_input)
            d = LeakyReLU(alpha=0.2)(d)
            if normalization:
                d = InstanceNormalization()(d)
            return d

        img = Input(shape=self.img_shape)

        d1 = d_layer(img, self.df, normalization=False)
        d2 = d_layer(d1, self.df*2)
        d3 = d_layer(d2, self.df*4)
        d4 = d_layer(d3, self.df*8)

        validity = Conv2D(1, kernel_size=4, strides=1, padding='same')(d4)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, sample_interval=50,load=False):

        start_time = datetime.datetime.now()


        if load == True:
            1+1
        # Adversarial loss ground truths
        valid = np.ones((batch_size,) + self.disc_patch)
        fake = np.zeros((batch_size,) + self.disc_patch)
        
        
        Gloss=100
        for epoch in range(epochs):

            for batch_i, (imgs_A, imgs_B) in enumerate(self.data_loader.load_batch(batch_size)):

                # ----------------------
                #  Train Discriminators
                # ----------------------

                # Translate images to opposite domain
                fake_B = self.g_AB.predict(imgs_A)
                fake_A = self.g_BA.predict(imgs_B)

                # Train the discriminators (original images = real / translated = Fake)
                dA_loss_real = self.d_A.train_on_batch(imgs_A, valid)
                dA_loss_fake = self.d_A.train_on_batch(fake_A, fake)
                dA_loss = 0.5 * np.add(dA_loss_real, dA_loss_fake)

                dB_loss_real = self.d_B.train_on_batch(imgs_B, valid)
                dB_loss_fake = self.d_B.train_on_batch(fake_B, fake)
                dB_loss = 0.5 * np.add(dB_loss_real, dB_loss_fake)

                # Total disciminator loss
                d_loss = 0.5 * np.add(dA_loss, dB_loss)

                # ------------------
                #  Train Generators
                # ------------------

                # Train the generators
                g_loss = self.combined.train_on_batch([imgs_A, imgs_B], [valid, valid, \
                                                                         imgs_B, imgs_A, \
                                                                         imgs_A, imgs_B])

                elapsed_time = datetime.datetime.now() - start_time
                # Plot the progress
                print ("[%d] [%d/%d] time: %s, [d_loss: %f, g_loss: %f]" % (epoch, batch_i,
                                                                        self.data_loader.n_batches,
                                                                        elapsed_time,
                                                                        d_loss[0], g_loss[0]))

                
                # If at save interval => save generated image samples
                if batch_i % 4 == 0 and epoch % 10 == 0  :
                    self.sample_images(epoch, batch_i)
                if g_loss[0] < Gloss :
                    self.g_AB.save('./models/gen_model.h5')
                    self.g_AB.save_weights('./models/gen_weights.h5')
                    self.combined.save('./models/model.h5')
                    self.combined.save_weights('./models/weights.h5')
                    Gloss = g_loss[0] 

    def sample_images(self, epoch, batch_i):
        os.makedirs('images/%s' % self.dataset_name, exist_ok=True)
        r, c = 2, 3

        imgs_A, imgs_B = self.data_loader.load_data(batch_size=1, is_testing=True)

        # Translate images to the other domain
        fake_B = self.g_AB.predict(imgs_A)
        fake_A = self.g_BA.predict(imgs_B)
        # Translate back to original domain
        reconstr_A = self.g_BA.predict(fake_B)
        reconstr_B = self.g_AB.predict(fake_A)

        gen_imgs = np.concatenate([imgs_A, fake_B, reconstr_A, imgs_B, fake_A, reconstr_B])

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        titles = ['Original', 'Translated', 'Reconstructed']
        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt])
                axs[i, j].set_title(titles[j])
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/%s/%d_%d.png" % (self.dataset_name, epoch, batch_i))
        plt.close()

def signal_handler(self,sig, frame):
        print('Graceful Interupt')


In [3]:

if __name__ == '__main__':


    gan = colorizeSketch()
    
    gan.train(epochs=100, batch_size=3, sample_interval=100,load=False)


C:\Users\me\Anaconda3\envs\tf-gpu\lib\site-packages\keras\engine\training.py:478: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


[0] [0/278] time: 0:01:33.836795, [d_loss: 28.354725, g_loss: 112.704742]
[0] [1/278] time: 0:01:35.724061, [d_loss: 32.449013, g_loss: 136.212524]
[0] [2/278] time: 0:01:36.959289, [d_loss: 35.028946, g_loss: 15.577663]
[0] [3/278] time: 0:01:53.948734, [d_loss: 6.908571, g_loss: 4.905550]
[0] [4/278] time: 0:02:00.398747, [d_loss: 1.528908, g_loss: 4.761908]
[0] [5/278] time: 0:02:07.570089, [d_loss: 0.595515, g_loss: 4.724053]
[0] [6/278] time: 0:02:14.203060, [d_loss: 0.619705, g_loss: 5.112312]
[0] [7/278] time: 0:02:15.443563, [d_loss: 0.455823, g_loss: 5.712058]
[0] [8/278] time: 0:02:16.687861, [d_loss: 0.568342, g_loss: 4.567587]
[0] [9/278] time: 0:02:24.118075, [d_loss: 0.769455, g_loss: 5.510798]
[0] [10/278] time: 0:02:25.389131, [d_loss: 0.333366, g_loss: 4.286175]
[0] [11/278] time: 0:02:34.324605, [d_loss: 0.215233, g_loss: 4.383053]
[0] [12/278] time: 0:02:35.538776, [d_loss: 0.204584, g_loss: 4.598567]
[0] [13/278] time: 0:02:37.123456, [d_loss: 0.414084, g_loss: 3.65

[0] [114/278] time: 0:05:37.449352, [d_loss: 0.304545, g_loss: 2.930466]
[0] [115/278] time: 0:05:38.664726, [d_loss: 0.274727, g_loss: 2.841469]
[0] [116/278] time: 0:05:39.859530, [d_loss: 0.356559, g_loss: 2.760008]
[0] [117/278] time: 0:05:41.409580, [d_loss: 0.400938, g_loss: 2.454463]
[0] [118/278] time: 0:05:42.590140, [d_loss: 0.362055, g_loss: 2.657859]
[0] [119/278] time: 0:05:43.768194, [d_loss: 0.417021, g_loss: 2.596565]
[0] [120/278] time: 0:05:44.944886, [d_loss: 0.445773, g_loss: 2.272656]
[0] [121/278] time: 0:05:52.222867, [d_loss: 0.275516, g_loss: 2.634515]
[0] [122/278] time: 0:05:53.410004, [d_loss: 0.433833, g_loss: 1.870053]
[0] [123/278] time: 0:06:00.321714, [d_loss: 0.410387, g_loss: 2.347584]
[0] [124/278] time: 0:06:01.500509, [d_loss: 0.334453, g_loss: 2.604024]
[0] [125/278] time: 0:06:03.027350, [d_loss: 0.394861, g_loss: 2.458895]
[0] [126/278] time: 0:06:04.218114, [d_loss: 0.280226, g_loss: 2.788971]
[0] [127/278] time: 0:06:05.404941, [d_loss: 0.3417

[0] [227/278] time: 0:08:24.753530, [d_loss: 0.364168, g_loss: 2.156168]
[0] [228/278] time: 0:08:25.940679, [d_loss: 0.366646, g_loss: 2.139067]
[0] [229/278] time: 0:08:27.506173, [d_loss: 0.300132, g_loss: 2.148498]
[0] [230/278] time: 0:08:28.697938, [d_loss: 0.405020, g_loss: 1.914601]
[0] [231/278] time: 0:08:29.884872, [d_loss: 0.427204, g_loss: 2.428478]
[0] [232/278] time: 0:08:31.081628, [d_loss: 0.412047, g_loss: 2.029418]
[0] [233/278] time: 0:08:32.655611, [d_loss: 0.395739, g_loss: 1.890754]
[0] [234/278] time: 0:08:33.855844, [d_loss: 0.295208, g_loss: 2.153497]
[0] [235/278] time: 0:08:35.057948, [d_loss: 0.357456, g_loss: 1.761273]
[0] [236/278] time: 0:08:36.255035, [d_loss: 0.327927, g_loss: 1.572009]
[0] [237/278] time: 0:08:37.827123, [d_loss: 0.309298, g_loss: 2.033520]
[0] [238/278] time: 0:08:39.019868, [d_loss: 0.351423, g_loss: 1.882016]
[0] [239/278] time: 0:08:40.208267, [d_loss: 0.293620, g_loss: 2.422067]
[0] [240/278] time: 0:08:41.389764, [d_loss: 0.5398

[1] [64/278] time: 0:10:45.551186, [d_loss: 0.275811, g_loss: 1.878384]
[1] [65/278] time: 0:10:46.745234, [d_loss: 0.331749, g_loss: 1.718245]
[1] [66/278] time: 0:10:47.928421, [d_loss: 0.288403, g_loss: 2.012295]
[1] [67/278] time: 0:10:49.115288, [d_loss: 0.319925, g_loss: 1.916503]
[1] [68/278] time: 0:10:50.300656, [d_loss: 0.285312, g_loss: 2.065731]
[1] [69/278] time: 0:10:51.481742, [d_loss: 0.500041, g_loss: 1.661636]
[1] [70/278] time: 0:10:52.665417, [d_loss: 0.241291, g_loss: 2.214079]
[1] [71/278] time: 0:10:53.841977, [d_loss: 0.353513, g_loss: 1.744513]
[1] [72/278] time: 0:10:55.028047, [d_loss: 0.277953, g_loss: 1.810024]
[1] [73/278] time: 0:10:56.217139, [d_loss: 0.306336, g_loss: 2.156269]
[1] [74/278] time: 0:10:57.399378, [d_loss: 0.372581, g_loss: 1.881238]
[1] [75/278] time: 0:10:58.578952, [d_loss: 0.446089, g_loss: 2.126729]
[1] [76/278] time: 0:10:59.759605, [d_loss: 0.445019, g_loss: 1.677724]
[1] [77/278] time: 0:11:00.942823, [d_loss: 0.385388, g_loss: 1.

...

[98] [54/278] time: 9:11:21.471501, [d_loss: 0.060724, g_loss: 2.269544]
[98] [55/278] time: 9:11:22.672786, [d_loss: 0.055834, g_loss: 2.356460]
[98] [56/278] time: 9:11:23.869297, [d_loss: 0.102438, g_loss: 2.092184]
[98] [57/278] time: 9:11:25.069939, [d_loss: 0.088667, g_loss: 2.275519]
[98] [58/278] time: 9:11:26.270596, [d_loss: 0.067110, g_loss: 2.263865]
[98] [59/278] time: 9:11:27.471666, [d_loss: 0.092487, g_loss: 2.325741]
[98] [60/278] time: 9:11:28.683072, [d_loss: 0.060043, g_loss: 2.503334]
[98] [61/278] time: 9:11:29.872998, [d_loss: 0.115949, g_loss: 2.116074]
[98] [62/278] time: 9:11:31.071267, [d_loss: 0.057117, g_loss: 2.277780]
[98] [63/278] time: 9:11:32.271282, [d_loss: 0.096184, g_loss: 2.323889]
[98] [64/278] time: 9:11:33.467948, [d_loss: 0.066672, g_loss: 2.528032]
[98] [65/278] time: 9:11:34.667909, [d_loss: 0.053120, g_loss: 2.321780]
[98] [66/278] time: 9:11:35.868094, [d_loss: 0.053990, g_loss: 2.451326]
[98] [67/278] time: 9:11:37.066699, [d_loss: 0.0729

[98] [166/278] time: 9:13:35.744963, [d_loss: 0.130963, g_loss: 2.975448]
[98] [167/278] time: 9:13:36.945077, [d_loss: 0.225265, g_loss: 2.458994]
[98] [168/278] time: 9:13:38.139378, [d_loss: 0.134759, g_loss: 2.484551]
[98] [169/278] time: 9:13:39.332153, [d_loss: 0.096328, g_loss: 2.163767]
[98] [170/278] time: 9:13:40.525251, [d_loss: 0.125792, g_loss: 2.889049]
[98] [171/278] time: 9:13:41.717421, [d_loss: 0.073162, g_loss: 2.407095]
[98] [172/278] time: 9:13:42.918899, [d_loss: 0.112435, g_loss: 2.967512]
[98] [173/278] time: 9:13:44.112762, [d_loss: 0.103601, g_loss: 2.236790]
[98] [174/278] time: 9:13:45.316835, [d_loss: 0.073146, g_loss: 2.484778]
[98] [175/278] time: 9:13:46.518289, [d_loss: 0.108980, g_loss: 3.073240]
[98] [176/278] time: 9:13:47.721087, [d_loss: 0.050305, g_loss: 2.248932]
[98] [177/278] time: 9:13:48.922990, [d_loss: 0.064757, g_loss: 3.005936]
[98] [178/278] time: 9:13:50.119603, [d_loss: 0.100230, g_loss: 2.367089]
[98] [179/278] time: 9:13:51.314671, [

[99] [0/278] time: 9:15:48.707380, [d_loss: 0.079455, g_loss: 2.500995]
[99] [1/278] time: 9:15:49.898283, [d_loss: 0.090792, g_loss: 2.268512]
[99] [2/278] time: 9:15:51.093846, [d_loss: 0.118619, g_loss: 2.299181]
[99] [3/278] time: 9:15:52.287669, [d_loss: 0.080179, g_loss: 2.424040]
[99] [4/278] time: 9:15:53.479556, [d_loss: 0.097565, g_loss: 2.490093]
[99] [5/278] time: 9:15:54.674631, [d_loss: 0.098736, g_loss: 2.390090]
[99] [6/278] time: 9:15:55.872747, [d_loss: 0.093876, g_loss: 2.645494]
[99] [7/278] time: 9:15:57.067688, [d_loss: 0.126380, g_loss: 2.198645]
[99] [8/278] time: 9:15:58.274302, [d_loss: 0.058257, g_loss: 2.450475]
[99] [9/278] time: 9:15:59.471360, [d_loss: 0.062292, g_loss: 2.330500]
[99] [10/278] time: 9:16:00.670425, [d_loss: 0.145687, g_loss: 2.240913]
[99] [11/278] time: 9:16:01.868789, [d_loss: 0.095361, g_loss: 2.313217]
[99] [12/278] time: 9:16:03.057633, [d_loss: 0.066767, g_loss: 2.473032]
[99] [13/278] time: 9:16:04.256424, [d_loss: 0.142435, g_loss

[99] [113/278] time: 9:18:04.114605, [d_loss: 0.074660, g_loss: 2.836251]
[99] [114/278] time: 9:18:05.310691, [d_loss: 0.113096, g_loss: 2.539806]
[99] [115/278] time: 9:18:06.501046, [d_loss: 0.049066, g_loss: 2.390661]
[99] [116/278] time: 9:18:07.707121, [d_loss: 0.083247, g_loss: 2.297035]
[99] [117/278] time: 9:18:08.901013, [d_loss: 0.116661, g_loss: 2.351754]
[99] [118/278] time: 9:18:10.103830, [d_loss: 0.102394, g_loss: 2.305712]
[99] [119/278] time: 9:18:11.300694, [d_loss: 0.054756, g_loss: 2.518381]
[99] [120/278] time: 9:18:12.493563, [d_loss: 0.107630, g_loss: 2.421216]
[99] [121/278] time: 9:18:13.685492, [d_loss: 0.143612, g_loss: 2.404706]
[99] [122/278] time: 9:18:14.884624, [d_loss: 0.109545, g_loss: 2.113303]
[99] [123/278] time: 9:18:16.079328, [d_loss: 0.126899, g_loss: 2.161546]
[99] [124/278] time: 9:18:17.280807, [d_loss: 0.111691, g_loss: 2.499283]
[99] [125/278] time: 9:18:18.476321, [d_loss: 0.135287, g_loss: 2.305700]
[99] [126/278] time: 9:18:19.669844, [

[99] [224/278] time: 9:20:17.176629, [d_loss: 0.045419, g_loss: 2.482744]
[99] [225/278] time: 9:20:18.370099, [d_loss: 0.077900, g_loss: 2.672935]
[99] [226/278] time: 9:20:19.561835, [d_loss: 0.074184, g_loss: 2.230450]
[99] [227/278] time: 9:20:20.763063, [d_loss: 0.099049, g_loss: 3.226039]
[99] [228/278] time: 9:20:21.960582, [d_loss: 0.072294, g_loss: 2.543547]
[99] [229/278] time: 9:20:23.166206, [d_loss: 0.067495, g_loss: 2.700533]
[99] [230/278] time: 9:20:24.359692, [d_loss: 0.092779, g_loss: 2.318112]
[99] [231/278] time: 9:20:25.555345, [d_loss: 0.147742, g_loss: 1.894178]
[99] [232/278] time: 9:20:26.755283, [d_loss: 0.051396, g_loss: 2.293426]
[99] [233/278] time: 9:20:27.941741, [d_loss: 0.086555, g_loss: 2.517598]
[99] [234/278] time: 9:20:29.144365, [d_loss: 0.056271, g_loss: 2.362979]
[99] [235/278] time: 9:20:30.334830, [d_loss: 0.048854, g_loss: 2.332590]
[99] [236/278] time: 9:20:31.529895, [d_loss: 0.037321, g_loss: 2.372539]
[99] [237/278] time: 9:20:32.729595, [

In [ ]:

if __name__ == '__main__':


    gan = colorizeSketch()
    
    gan.train(epochs=100, batch_size=3, sample_interval=100,load=False)

